In [3]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

from collections import Counter

from sklearn.preprocessing import OneHotEncoder


from sklearn.model_selection import train_test_split

from keras import models
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
import streamlit as st

In [80]:
!pip install protobuf==3.20.*

ERROR: Could not find a version that satisfies the requirement protobuf==3.20.* (from versions: 2.0.0b0, 2.0.3, 2.3.0, 2.4.1, 2.5.0, 2.6.0, 2.6.1, 3.0.0a2, 3.0.0a3, 3.0.0b1, 3.0.0b1.post1, 3.0.0b1.post2, 3.0.0b2, 3.0.0b2.post1, 3.0.0b2.post2, 3.0.0b3, 3.0.0b4, 3.0.0, 3.1.0, 3.1.0.post1, 3.2.0rc1, 3.2.0rc1.post1, 3.2.0rc2, 3.2.0, 3.3.0, 3.4.0, 3.5.0.post1, 3.5.1, 3.5.2, 3.5.2.post1, 3.6.0, 3.6.1, 3.7.0rc2, 3.7.0rc3, 3.7.0, 3.7.1, 3.8.0rc1, 3.8.0, 3.9.0rc1, 3.9.0, 3.9.1, 3.9.2, 3.10.0rc1, 3.10.0, 3.11.0rc1, 3.11.0rc2, 3.11.0, 3.11.1, 3.11.2, 3.11.3, 3.12.0rc1, 3.12.0rc2, 3.12.0, 3.12.1, 3.12.2, 3.12.4, 3.13.0rc3, 3.13.0, 3.14.0rc1, 3.14.0rc2, 3.14.0rc3, 3.14.0, 3.15.0rc1, 3.15.0rc2, 3.15.0, 3.15.1, 3.15.2, 3.15.3, 3.15.4, 3.15.5, 3.15.6, 3.15.7, 3.15.8, 3.16.0rc1, 3.16.0rc2, 3.16.0, 3.17.0rc1, 3.17.0rc2, 3.17.0, 3.17.1, 3.17.2, 3.17.3, 3.18.0rc1, 3.18.0rc2, 3.18.0, 3.18.1, 3.19.0rc1, 3.19.0rc2, 3.19.0, 3.19.1, 3.19.2, 3.19.3, 3.19.4, 4.0.0rc1, 4.0.0rc2, 4.21.0rc1, 4.21.0rc2, 4.21.0)
ERRO

In [4]:
X_train = pd.read_csv('X_train.csv')

In [5]:
data = pd.read_csv('test_data_evaluation_part2.csv')
data = data.drop(data.columns[:2],axis = 1)
data['Nationality'] = data['Nationality'].str.lower() 
data['DistributionChannel'] = data['DistributionChannel'].str.replace(' ','_')
data['DistributionChannel'] = data['DistributionChannel'].str.replace('/','_')
data['DistributionChannel'] = data['DistributionChannel'].str.lower()
data['MarketSegment'] = data['MarketSegment'].str.replace(' ','_')
data['MarketSegment'] = data['MarketSegment'].str.replace('/','_')
data['MarketSegment'] = data['MarketSegment'].str.lower()
data['Age'].fillna(46.0,inplace = True)


In [6]:
lis = ['Nationality','DistributionChannel','MarketSegment']
dic = dict()
for i in lis:
    vectorizer = CountVectorizer()
    vectorizer.fit(X_train[i].values)
    dic[i] = vectorizer.transform(data[i].values)
    #print(dic[i].shape)

In [7]:
dic

{'Nationality': <1000x164 sparse matrix of type '<class 'numpy.int64'>'
 	with 1000 stored elements in Compressed Sparse Row format>,
 'DistributionChannel': <1000x4 sparse matrix of type '<class 'numpy.int64'>'
 	with 1000 stored elements in Compressed Sparse Row format>,
 'MarketSegment': <1000x7 sparse matrix of type '<class 'numpy.int64'>'
 	with 1000 stored elements in Compressed Sparse Row format>}

In [8]:
from sklearn.preprocessing import Normalizer
lisn = ['Age', 'DaysSinceCreation', 'AverageLeadTime',
       'LodgingRevenue', 'OtherRevenue', 'BookingsCanceled',
       'BookingsNoShowed', 'BookingsCheckedIn', 'PersonsNights',
       'RoomNights', 'DaysSinceLastStay', 'DaysSinceFirstStay', 'SRHighFloor',
       'SRLowFloor', 'SRAccessibleRoom', 'SRMediumFloor', 'SRBathtub',
       'SRShower', 'SRCrib', 'SRKingSizeBed', 'SRTwinBed',
       'SRNearElevator', 'SRAwayFromElevator', 'SRNoAlcoholInMiniBar',
       'SRQuietRoom']
for i in lisn:
    normalizer = Normalizer()
    normalizer.fit(X_train[i].values.reshape(-1,1))
    dic[i] = normalizer.transform(data[i].values.reshape(-1,1))
    #print(dic[i].shape)

In [9]:
(dic['Nationality'],dic['DistributionChannel'],dic['MarketSegment'],dic['Age'],dic['DaysSinceCreation'],dic['AverageLeadTime'],dic['LodgingRevenue'],dic['OtherRevenue'],dic['BookingsCanceled'],dic['BookingsNoShowed'],dic['BookingsCheckedIn'],dic['PersonsNights'],dic['RoomNights'],dic['DaysSinceLastStay'],dic['DaysSinceFirstStay'],dic['SRHighFloor'],dic['SRLowFloor'],dic['SRAccessibleRoom'],dic['SRMediumFloor'],dic['SRBathtub'],dic['SRShower'],dic['SRCrib'],dic['SRKingSizeBed'],dic['SRTwinBed'],dic['SRNearElevator'],dic['SRAwayFromElevator'],dic['SRNoAlcoholInMiniBar'],dic['SRQuietRoom'])
        

(<1000x164 sparse matrix of type '<class 'numpy.int64'>'
 	with 1000 stored elements in Compressed Sparse Row format>,
 <1000x4 sparse matrix of type '<class 'numpy.int64'>'
 	with 1000 stored elements in Compressed Sparse Row format>,
 <1000x7 sparse matrix of type '<class 'numpy.int64'>'
 	with 1000 stored elements in Compressed Sparse Row format>,
 array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
   

In [10]:
from scipy.sparse import hstack
query_data = hstack((dic['Nationality'],dic['DistributionChannel'],dic['MarketSegment'],dic['Age'],dic['DaysSinceCreation'],dic['AverageLeadTime'],dic['LodgingRevenue'],dic['OtherRevenue'],dic['BookingsCanceled'],dic['BookingsNoShowed'],dic['BookingsCheckedIn'],dic['PersonsNights'],dic['RoomNights'],dic['DaysSinceLastStay'],dic['DaysSinceFirstStay'],dic['SRHighFloor'],dic['SRLowFloor'],dic['SRAccessibleRoom'],dic['SRMediumFloor'],dic['SRBathtub'],dic['SRShower'],dic['SRCrib'],dic['SRKingSizeBed'],dic['SRTwinBed'],dic['SRNearElevator'],dic['SRAwayFromElevator'],dic['SRNoAlcoholInMiniBar'],dic['SRQuietRoom'])).tocsr()

In [11]:
query_data.shape

(1000, 200)

In [12]:
import keras

model = keras.models.load_model(r'C:\Users\goldr\Desktop\Nxtlab')

In [81]:
[1 if x>0.5 else 0 for x in model.predict(query_data)]

[0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
